In [1]:
import suffix_tree
import pandas as pd
import os
import json

In [2]:
class STreeEx():
    def __init__(self, sTree):
        # 傳入原套件後綴樹
        self.sTree = sTree

    def lrs(self):
        # 最深非葉節點
        deepestNode = self._find_lrs(self.sTree.root)
        start = deepestNode.idx
        end = deepestNode.idx + deepestNode.depth
        return self.sTree.word[start:end]

    def _find_lrs(self, node):
        nodes = [self._find_lrs(n)
            for (n,_) in node.transition_links
            # 排除葉節點
            if n.transition_links != []]

        if nodes == []:
            return node
        
        deepestNode = max(nodes, key=lambda n: n.depth)
        return deepestNode

# Input File

In [3]:
current_path = os.path.join(os.path.expanduser("~"), "jupyter", "Fix_DCADE_Output")
#current_path = os.path.join(os.path.expanduser("~"), "Desktop", "DCADE", "TAM")
input_file_path = os.path.join(current_path, "Output", "TXT", "TableA.txt")
print(input_file_path)

C:\Users\rick\jupyter\Fix_DCADE_Output\Output\TXT\TableA.txt


In [4]:
content = []
recb_start = 0
recb_end = 0
tag = []
ids = []
classes = []
pathid = []
parentid = []
tecid = []
cecid = []
encoding = []
col = []
others = []
#with open(input_file_path, encoding='utf-8') as file:
with open(input_file_path) as file:
    line = file.readline()
    while(line):
        line = file.readline()
        tmp = line.split('\t')
        if tmp[0] == "Content":
            content = tmp[1:-1]
        elif tmp[0] == "RecB":
            tmp = tmp[1:-1]
            for index in range(len(tmp)):
                if tmp[index] == "SDR":
                    recb_start = index
                elif tmp[index] == "EDR":
                    recb_end = index
        elif tmp[0] == "Tag":
            tag = tmp[1:-1]
        elif tmp[0] == "ID":
            ids = tmp[1:-1]
        elif tmp[0] == "Class":
            classes = tmp[1:-1]
        elif tmp[0] == "PathId":
            pathid = tmp[1:-1]
        elif tmp[0] == "ParentId":
            parentid = tmp[1:-1]
        elif tmp[0] == "TECId":
            tecid = tmp[1:-1]
        elif tmp[0] == "CECId":
            cecid = tmp[1:-1]
        elif tmp[0] == "Encoding":
            encoding = tmp[1:-1]
        elif tmp[0] == "Col Type":
            col = tmp[1:-1]
        else:
            others.append(tmp[1:-1])
others = others[0:-4]

In [5]:
tec_dict = {}
unique_mt = []
for node in range(len(col)):
    if col[node] == 'MT':
        if tecid[node] not in tec_dict.keys():
            tec_dict[tecid[node]] = [node]
        else:
            tec_dict[tecid[node]].append(node)
for key in tec_dict.keys():
    if len(tec_dict[key]) == 1:
        unique_mt += tec_dict[key]
print("Unique MT's index:\n", unique_mt)

Unique MT's index:
 [3, 4, 5, 6, 8, 10, 16, 18, 19, 20, 22, 23, 25, 28, 81, 136, 279, 281, 284, 291, 303, 306, 308]


# Translate node in to unicode for suffix tree

Change `encode` to use different attribute to encode leafnodes.
- 0: **Pathid** + **Parentid** + **TECid** + **CECid** + **Encoding** + **Column Type**
- 1: **Id** + **Class** + **Pathid** + **CECid**
- 2: **Pathid** + **Parentid** + **CECid**
<br><br>
Leafnodes will have same encoding if they have same attribute.

In [6]:
encode = 3
#encode_option = [1,0,0,0,1,0,0,1,0]
encode_option = [0,0,0,1,1,0,1,0,0]
encode_col = [tag, ids, classes, pathid, parentid, tecid, cecid, encoding, col]
whole_string = ""
test = ""
node_dict = {} # code -> node num
code_dict = {} # code -> content
index_dict = {} # code -> first index
index = 0
for node in range(len(pathid)):
    """if encode == 0:
        code = pathid[node] + "," + parentid[node] + "," + tecid[node] + "," + cecid[node] + "," + encoding[node] + "," + col[node]
    elif encode == 1:
        code = ids[node] + "," + classes[node] + "," + pathid[node] + "," + cecid[node]
    elif encode == 2:
        code = pathid[node] + "," + parentid[node] + "," + cecid[node]
    elif encode == 3:
        code = pathid[node] + "," + parentid[node] + "," + cecid[node] + "," + encoding[node]"""
    code = ""
    for col_num in range(9):
        if encode_option[col_num] == 1:
            code += encode_col[col_num][node]
    #if 'O' in col[node]: continue
    if code not in node_dict.keys():
        node_dict[code] = index
        code_dict[code] = others[0][node]
        index_dict[code] = node
        whole_string += chr(index)
        index += 1
        if index == 45 or index == 32:
            index+=1
    else:
        whole_string += chr(node_dict[code])
print("Example: ")
"""if encode == 0:
    print(pathid[0] + "," + parentid[0] + "," + tecid[0] + "," + cecid[0] + "," + encoding[0] + "," + col[0])
elif encode == 1:
    print(ids[0] + "," + classes[0] + "," + pathid[0] + "," + cecid[0])
elif encode == 2:
    print(pathid[0] + "," + parentid[0] + "," + cecid[0])
if encode == 3:
    print(pathid[0] + "," + parentid[0] + "," + cecid[0] + "," + encoding[0])"""

for col_num in range(9):
    if encode_option[col_num] == 1:
        print(encode_col[col_num][node], end='')
inv_node_dict = {v: k for k, v in node_dict.items()} # node num -> code
inv_index_dict = {v: k for k, v in index_dict.items()} # index num -> code
print("\nConvert to Unicode String:\n", whole_string)

Example: 
95:[1]9-185
Convert to Unicode String:
  	
!"#$%&'()*!"#$%&'()*+,./.0.1.2.3.4.5.6.7.8.9.:.;.<.=.>.?.@.A.B.C.D.E.F.G.H.I.JJKLLMKNKOPJJKLLQNKOKJJKLLMKNKOKJJKLLMKNKOPJJKLLMQNKOKJJKLLMKNKOKJJKLLMKNKOKJJKLLMKNKOKJJKLLKNKOPJJKLLMKNKOPJJKLLMQNKOKJJKLLKNKOKRSTUVWXYZ[\]^_`abcdefghijklmnopqrst


In [7]:
segments = []
for seg_point in range(len(unique_mt)):
    if seg_point != len(unique_mt)-1:
        tmp = whole_string[unique_mt[seg_point]+1:unique_mt[seg_point+1]]
        if len(tmp) > 10:
            segments.append(tmp)
segments = list(filter(None, segments))
print(len(segments))
print(segments)

4
['\x1c\x1d\x1e\x1d\x1f\x1d!\x1d"\x1d#\x1d$\x1d%\x1d&\x1d\'\x1d(\x1d)\x1d*\x1d\x1c\x1d\x1e\x1d\x1f\x1d!\x1d"\x1d#\x1d$\x1d%\x1d&\x1d\'\x1d(\x1d)\x1d*\x1d', ',./.0.1.2.3.4.5.6.7.8.9.:.;.<.=.>.?.@.A.B.C.D.E.F.G.H.', '.JJKLLMKNKOPJJKLLQNKOKJJKLLMKNKOKJJKLLMKNKOPJJKLLMQNKOKJJKLLMKNKOKJJKLLMKNKOKJJKLLMKNKOKJJKLLKNKOPJJKLLMKNKOPJJKLLMQNKOKJJKLLKNKOKR\x0cS\x0cT\x0cU\x0cV\x0cW\x0c', 'defghijklmn']


In [8]:
def find_all_indexes(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1

# Show maximal repeats

Ignore maximal repeats if length equal to 1.

In [9]:
record_seg = []
trees = [suffix_tree.Tree({'A': seg}) for seg in segments]
for tree_idx in range(len(trees)):
    tree = trees[tree_idx]
    #print("Total Maximal repeats: ", len(tree.maximal_repeats()), '\n')
    group = {}
    top_repeats = (0, '')
    for C, path in tree.maximal_repeats():
        if len(path) == 1: continue
        count = 0
        #print("Length: ", len(path), "Pattern: {", path, "}")
        for id_, path2 in tree.find_all (path):
            count += 1
        #print("Count: ", count)
        if count > top_repeats[0]:
            top_repeats = (count, str(path).replace(' ', ''))
        """for i in path:
            print("Content: ", code_dict[inv_node_dict[ord(i)]])"""
        if index_dict[inv_node_dict[ord(path[0])]] not in group.keys():
            group[index_dict[inv_node_dict[ord(path[0])]]] = []
            group[index_dict[inv_node_dict[ord(path[0])]]].append(path)
        else:
            group[index_dict[inv_node_dict[ord(path[0])]]].append(path)
        #print("\n", "="*50, "\n")
    if top_repeats[0] != 0:
        record_seg.append((tree_idx, top_repeats[1]))
        print(top_repeats)

(2, '\x1c\x1d\x1e\x1d\x1f\x1d!\x1d"\x1d#\x1d$\x1d%\x1d&\x1d\'\x1d(\x1d)\x1d*\x1d')
(12, 'JJKLL')


# Whole String split by top repeats

In [10]:
max_len = []
all_seqs = []
for seg_idx in range(len(record_seg)):
    max_len.append(0)
    seqs = []
    pos = find_all_indexes(segments[record_seg[seg_idx][0]], record_seg[seg_idx][1])
    for idx in range(len(pos)):
        if idx+1 != len(pos):
            seqs.append(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx+1]])
            if len(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx+1]]) > max_len[seg_idx]:
                max_len[seg_idx] = len(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx+1]])
        else:
            seqs.append(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx]+max_len[seg_idx]+4])
    all_seqs.append(seqs)
print(all_seqs)

[['\x1c\x1d\x1e\x1d\x1f\x1d!\x1d"\x1d#\x1d$\x1d%\x1d&\x1d\'\x1d(\x1d)\x1d*\x1d', '\x1c\x1d\x1e\x1d\x1f\x1d!\x1d"\x1d#\x1d$\x1d%\x1d&\x1d\'\x1d(\x1d)\x1d*\x1d'], ['JJKLLMKNKOP', 'JJKLLQNKOK', 'JJKLLMKNKOK', 'JJKLLMKNKOP', 'JJKLLMQNKOK', 'JJKLLMKNKOK', 'JJKLLMKNKOK', 'JJKLLMKNKOK', 'JJKLLKNKOP', 'JJKLLMKNKOP', 'JJKLLMQNKOK', 'JJKLLKNKOKR\x0cS\x0cT']]


# MSA

In [11]:
import cstar
removed_whole_string = whole_string
for seg_idx in range(len(all_seqs)):
    json_result = []
    print("First round MSA\n", "="*100)
    scores = [1, -1, -3] # matchScore, mismatchScore, gapScore
    if len(all_seqs[seg_idx][:-1]) == 1:
        msa = all_seqs[seg_idx][:-1]
    else:
        msa = cstar.CenterStar(scores, all_seqs[seg_idx][:-1]).msa()
    trans_dict = {}
    last_c = '-'
    end_idx = 0

    for i in range(len(msa)):
        if msa[i][-1] != '-' and last_c not in trans_dict.keys():
            last_c = msa[i][-1]
            trans_dict[last_c] = msa[i].replace('-', '')
        if msa[i].replace('-', '') not in trans_dict.keys():
            trans_dict[msa[i].replace('-', '')] = msa[i]
        else: pass
    for i in all_seqs[seg_idx][:-1]:
        print(i, "\n\t\t-> ", trans_dict[i])

    print('='*100, "\nSecond round MSA\n")
    msa_2 = cstar.CenterStar(scores, msa+[all_seqs[seg_idx][-1]]).msa()
    trans_dict_2 = {}

    for i in range(len(msa_2)):
        if msa_2[i].replace('-', '') not in trans_dict_2.keys():
            trans_dict_2[msa_2[i].replace('-', '')] = msa_2[i]
        else: pass
    for idx in range(len(trans_dict_2[trans_dict[last_c]])):
        if trans_dict_2[trans_dict[last_c]][idx] == last_c:
            end_idx = idx
    for i in all_seqs[seg_idx]:
        trans_dict_2[i] = trans_dict_2[i][:end_idx+1]
        tmp = trans_dict_2[i][:end_idx+1].replace('-', '')
        removed_whole_string = removed_whole_string.replace(tmp, '-'*len(tmp))
        print(i, "\n\t\t-> ", trans_dict_2[i])
    
    json_schema = [{} for i in range(len(trans_dict_2[list(trans_dict_2.keys())[0]]))]
    schema_check = [0 for i in range(len(trans_dict_2[list(trans_dict_2.keys())[0]]))]

    with open(os.path.join(current_path, "Output", "Set-" + str(seg_idx) + ".txt"), 'w', encoding='utf-8') as file:
        for page in range(len(others)):
            json_page = []
            output_dict = {} # Record which pattern is used
            for s in range(len(all_seqs[seg_idx])):
                json_set = []
                file.write(str(page) + "-" + str(seg_idx) + "-" + str(s) + "\t")
                tmp = find_all_indexes(whole_string, all_seqs[seg_idx][s])
                if all_seqs[seg_idx][s] not in output_dict.keys():
                    output_dict[all_seqs[seg_idx][s]] = 0
                else:
                    output_dict[all_seqs[seg_idx][s]] += 1
                #print("start:", tmp[output_dict[seqs[s]]], others[page][tmp[output_dict[seqs[s]]]])
                idx = 0
                for c in range(len(trans_dict_2[all_seqs[seg_idx][s]])):
                    if trans_dict_2[all_seqs[seg_idx][s]][c] == '-':
                        file.write('\t')
                        json_set.append('')
                    else:
                        file.write(others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx][:others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx].find(" ::")] + "\t")
                        json_set.append(others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx][:others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx].find(" ::")])
                        if schema_check[c] == 0:
                            schema_check[c] = 1
                            json_schema[c]["PathId"] = pathid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                            json_schema[c]["ParentId"] = parentid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx].split(':')[0].replace('\"', '')
                            json_schema[c]["Encoding"] = int(encoding[tmp[output_dict[all_seqs[seg_idx][s]]]+idx])
                            json_schema[c]["CECId"] = cecid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                            json_schema[c]["TECId"] = tecid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                            json_schema[c]["ColType"] = col[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                        idx += 1
                json_page.append(json_set)
                file.write('\n')
            json_result.append(json_page)
    with open(os.path.join(current_path, "Output", "Set-" + str(seg_idx) + "_schema.json"), 'w') as json_file:
        json.dump(json_result, json_file)
    with open(os.path.join(current_path, "Output", "Set-" + str(seg_idx) + "_schema.json"), 'w') as json_file:
        json.dump(json_schema, json_file)

First round MSA
!"#$%&'()* 
		->  !"#$%&'()*
Second round MSA

!"#$%&'()* 
		->  !"#$%&'()*
!"#$%&'()* 
		->  !"#$%&'()*
First round MSA
JJKLLMKNKOP 
		->  JJKLLMKNKOP
JJKLLQNKOK 
		->  JJKLL-QNKOK
JJKLLMKNKOK 
		->  JJKLLMKNKOK
JJKLLMKNKOP 
		->  JJKLLMKNKOP
JJKLLMQNKOK 
		->  JJKLLMQNKOK
JJKLLMKNKOK 
		->  JJKLLMKNKOK
JJKLLMKNKOK 
		->  JJKLLMKNKOK
JJKLLMKNKOK 
		->  JJKLLMKNKOK
JJKLLKNKOP 
		->  JJKLL-KNKOP
JJKLLMKNKOP 
		->  JJKLLMKNKOP
JJKLLMQNKOK 
		->  JJKLLMQNKOK
Second round MSA

JJKLLMKNKOP 
		->  JJKLLMKNKOP
JJKLLQNKOK 
		->  JJKLL-QNKOK
JJKLLMKNKOK 
		->  JJKLLMKNKOK
JJKLLMKNKOP 
		->  JJKLLMKNKOP
JJKLLMQNKOK 
		->  JJKLLMQNKOK
JJKLLMKNKOK 
		->  JJKLLMKNKOK
JJKLLMKNKOK 
		->  JJKLLMKNKOK
JJKLLMKNKOK 
		->  JJKLLMKNKOK
JJKLLKNKOP 
		->  JJKLL-KNKOP
JJKLLMKNKOP 
		->  JJKLLMKNKOP
JJKLLMQNKOK 
		->  JJKLLMQNKOK
JJKLLKNKOKRST 
		->  JJKLL-KNKOK


# Modified TableA Output

In [12]:
json_table = []
json_schema = []
for page in range(len(others)):
    json_page = []
    set_count = 0
    check = False
    for node in range(len(removed_whole_string)):
        schema_dict = {}
        if removed_whole_string[node] == '-':
            if check == False:
                check = True
                set_count += 1
                json_page.append(str(set_count) + '-' + str(page))
                schema_dict["PathId"] = ""
                schema_dict["ParentId"] = ""
                schema_dict["Encoding"] = -1
                schema_dict["CECId"] = ""
                schema_dict["TECId"] = ""
                schema_dict["ColType"] = "MR"
                json_schema.append(schema_dict)
            else: pass
        else:
            if check == True:
                check = False
            json_page.append(others[page][node][:others[page][node].find(" ::")])
            schema_dict["PathId"] = pathid[node]
            schema_dict["ParentId"] = parentid[node].split(':')[0].replace('\"', '')
            schema_dict["Encoding"] = int(encoding[node])
            schema_dict["CECId"] = cecid[node]
            schema_dict["TECId"] = tecid[node]
            schema_dict["ColType"] = col[node]
            json_schema.append(schema_dict)
    json_table.append(json_page)
with open(os.path.join(current_path, "Output", "TableA.json"), 'w') as json_file:
    json.dump(json_table, json_file)
with open(os.path.join(current_path, "Output", "SchemaTableA.json"), 'w') as json_file:
    json.dump(json_schema, json_file)